In [20]:
import enum
import logging
import os
import sys
import errno
import time
import math
import torch
from dataclasses import dataclass, field
from itertools import chain
from typing import Optional
from tqdm import tqdm

from transformers import BertTokenizer, BertForMaskedLM, AdamW
import torch
import torch.optim as optim
from torch.utils.data import DataLoader 

import argparse
from src.genie.data_module import RAMSDataModule

from src.genie.data import IEDataset, my_collate

In [21]:
output_dir = 'out'
with_tracking = False

batch_size = 64
n_epochs = 1
weight_decay = 0.0
learning_rate = 5e-5

lr_scheduler_type = "linear"
num_warmup_steps = 0.0
max_train_steps = 10_000
gradient_accumulation_steps = 1
with_tracking = False

In [22]:
train = IEDataset('preprocessed_data/train_RAMS_oracle.jsonl')

In [23]:
train = DataLoader(train, 
    pin_memory=True, num_workers=2, 
    collate_fn=my_collate,
    batch_size=16, 
    shuffle=True)

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_tokens(['<args>'])

model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(30523, 768)

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(n_epochs):
    for idx, data in tqdm(enumerate(train)):
        optimizer.zero_grad()
        input_ids = data['input_token_ids']
        labels = data['tgt_token_ids']
        attention_mask = data['input_attn_mask']
        loss, logits = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss.backward()
        optimizer.step()
    
        print(loss.item())


0it [00:00, ?it/s]

In [29]:
outputs.backward()